# MNIST with CNN

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.utils import make_grid

## Load Dataset

In [3]:
transform = transforms.ToTensor()

In [4]:
train_data = datasets.MNIST(root="/Dataset", train=True, download=True, transform=transform)

100%|██████████| 9912422/9912422 [00:00<00:00, 78690460.29it/s]


Extracting /Dataset/MNIST/raw/train-images-idx3-ubyte.gz to /Dataset/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 82856151.73it/s]


Extracting /Dataset/MNIST/raw/train-labels-idx1-ubyte.gz to /Dataset/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 21390964.67it/s]


Extracting /Dataset/MNIST/raw/t10k-images-idx3-ubyte.gz to /Dataset/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 13814741.67it/s]


Extracting /Dataset/MNIST/raw/t10k-labels-idx1-ubyte.gz to /Dataset/MNIST/raw



In [5]:
test_data = datasets.MNIST(root="/Dataset", train=False, download=True, transform=transform)

In [6]:
train_data

Dataset MNIST
    Number of datapoints: 60000
    Root location: /Dataset
    Split: Train
    StandardTransform
Transform: ToTensor()

In [7]:
test_data

Dataset MNIST
    Number of datapoints: 10000
    Root location: /Dataset
    Split: Test
    StandardTransform
Transform: ToTensor()

In [8]:
train_loader = DataLoader(train_data, batch_size=10, shuffle=True)

In [9]:
test_loader = DataLoader(test_data, batch_size=10, shuffle=False)

## Define Model

In [10]:
# 1 COLOR CHANNE, 6 FILTERS (OUTPUT CHANNELS), 3*3 KERNEL, STRIDE=1
conv1 = nn.Conv2d(1,6,3,1)   # ----> 6 filters, ---> pooling ---> conv2

# 6 Input Filters Conv1, 16 filters, 3*3, stride=1
conv2 = nn.Conv2d(6,16,3,1)

In [11]:
# First batch
for b,(X_train, y_train) in enumerate(train_data):
  break

In [12]:
X_train.shape

torch.Size([1, 28, 28])

In [13]:
y_train

5

In [14]:
x = X_train.view(1,1,28,28)  # ----> 4d batch (batch of 1 image)

In [15]:
x = F.relu(conv1(x))

In [16]:
x.shape

torch.Size([1, 6, 26, 26])

In [17]:
x = F.max_pool2d(x,2,2)

In [18]:
x.shape

torch.Size([1, 6, 13, 13])

In [19]:
x = F.relu(conv2(x))

In [20]:
x.shape

torch.Size([1, 16, 11, 11])

In [21]:
x = F.max_pool2d(x,2,2)

In [22]:
x.shape

torch.Size([1, 16, 5, 5])

In [23]:
x.view(-1,16*5*5).shape

torch.Size([1, 400])

In [24]:
x.shape

torch.Size([1, 16, 5, 5])

## Convolutional Neural Network Class

In [24]:
class ConvolutionalNeuralNetwork(nn.Module):

  def __init__(self):
    super().__init__()
    self.conv1 = nn.Conv2d(1, 6, 3, 1)
    self.conv2 = nn.Conv2d(6, 16, 3, 1)
    self.fc1 = nn.Linear(5*5*16, 120)
    self.fc2 = nn.Linear(120, 84)
    self.fc3 = nn.Linear(84, 10)
